# Import

In [ ]:
import numpy
import pandas as pd
import json
from konlpy.tag import Kkma, Twitter
from collections import Counter
import pytagcloud
import re
import gensim
import hanja

# 데이터 로드, 회의 발언과 정당 데이터 Join

In [ ]:
file_name = './pokr.json'
data = pd.read_json(file_name)
file_name2 = './member.json'
data2 = pd.read_json(file_name2)

In [ ]:
data = data[['content','person_id']]
data2 = data2[['person_id','party']]
data2['person_id'] = data2['person_id'].astype(float)

In [ ]:
merge_data = pd.merge(data, data2, how ='left', on='person_id')

# stop_words생성, 발언들을 형태소 단위로 나눠서 저장

In [ ]:
Tw = Twitter()
stop_words = set(['제','냐고','%','마니','까','까거','니까는','=','님','……','서','야','도','엔','가','감','수','요','시'])

In [ ]:
progress = [1,2,3,4] #진보 1,2,3,4
conser = [0,5,6] #보수 0,5,6

all_comment = [] #전체 발언
comment_pro = [] #진보 정당 발언
comment_con = [] #보수 정당 발언
        
for row in merge_data.iterrows():
    content = row[1]['content']
    party = row[1]['party']
    real_content = (re.sub('[ㆍ◎?,◯!.―()\n]', '', content))
    
    morphs = Tw.morphs(real_content)
    
    all_comment.append([word for word in (morphs) if word not in stop_words])
    
    if party in progress:
        comment_pro.append([word for word in morphs if word not in stop_words])
    if party in conser:
        comment_con.append([word for word in morphs if word not in stop_words])
    

In [ ]:
with open('all_comment.json', 'w') as fp:
    json.dump(all_comment, fp)
with open('comment_pro.json', 'w') as fp:
    json.dump(comment_pro, fp)
with open('comment_con.json', 'w') as fp:
    json.dump(comment_con, fp)

# Skip-gram 모델 생성, 차원은 200차원 10개 미만으로 나타난 단어는 무시

In [ ]:
all_commentmodel = gensim.models.Word2Vec(all_comment,size=200,min_count=10, sg=1)
model_con = gensim.models.Word2Vec(comment_con, size = 200, min_count=10,sg=1)
model_pro = gensim.models.Word2Vec(comment_pro,size = 200, min_count=10, sg=1) 

In [ ]:
all_commentmodel.most_similar('방범')

In [ ]:
model_pro.wv['국회']

In [ ]:
all_commentmodel.most_similar(positive=['새누리당', '야당'],negative=['민주당'])

In [ ]:
all_commentmodel.save('all_comment.json')
model_pro.save('comment_pro.json')
model_con.save('comment_con.json')

In [ ]:
'''
Word Cloud
all_comment_sum = []
for i in all_comment:
    all_comment_sum.extend(i)

count = Counter(all_comment_sum)
tag = (count.most_common(10))
taglist = pytagcloud.make_tags(tag, maxsize = 70)
pytagcloud.create_tag_image(taglist, 'wordcloud.jpg', size=(700,500), fontname="Korean", rectangular=False)

'''